<a href="https://colab.research.google.com/github/pannhapat/udacity_1001/blob/master/01_image_tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#package required by Keras
import tensorflow as tf
import numpy as np
import PIL

from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.applications.mobilenet import decode_predictions
from keras.preprocessing import image
from keras import backend as K


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def get_input_dim(input_layer):
  data_shape = input_layer.input_shape
  if data_shape[1] == 3: #channel first arrangement
    return (data_shape[2:4])
  else: #channel last arrangement
    return (data_shape[1:3])

def prepare_input(img_file,size): 
  img = image.load_img(img_file, target_size=size)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)
  return(preprocess_input(x))



In [0]:
my_model = MobileNet(include_top=True, weights='imagenet')
my_model.summary()
layer = my_model.layers[0]
layer.get_config()



W0824 06:03:14.732133 139701132642176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0824 06:03:14.754466 139701132642176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0824 06:03:14.761712 139701132642176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0824 06:03:14.786715 139701132642176 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0824 06:03:14.787689 1397011326

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

{'batch_input_shape': (None, 224, 224, 3),
 'dtype': 'float32',
 'name': 'input_1',
 'sparse': False}

In [0]:
!pip install iptcinfo3
from iptcinfo3 import IPTCInfo
import os

#keywords from IPTC need to convert to utf-8
convert_utf8 = lambda x: x.decode('utf-8')

img_path = '/content/drive/My Drive/photos/test_image_tagger'
img_list = os.listdir(img_path)

input_layer = my_model.layers[0]
input_sz = get_input_dim(input_layer)

for img in img_list:
  img_file = os.path.join(img_path,img)

  x = prepare_input(img_file,input_sz)
  preds = my_model.predict(x)
  decoded_preds = decode_predictions(preds, top=3)[0] #get top 3 prediction
  
  
  #retrieve keywords from image
  info = IPTCInfo(img_file)
  
  #init new keyword list with the existing one(s) retreived from the image 
  new_keywords = list(map(convert_utf8, info['keywords']))
  
  for prediction in decoded_preds:
    desc = prediction[1] #2nd element of the prediction is the description
    
    #only append the keyword if not already exist
    if desc not in new_keywords:
      new_keywords.append(desc)

  info['keywords'] = new_keywords
  status_okay = info.save()



